In [ ]:
import pickle
from matplotlib import pyplot as plt
import seaborn as sns
import json
import pandas as pd

In [ ]:
pickle_files = ["/gpfs/bbp.cscs.ch/data/scratch/proj16/magkanar/nmodl_llvm_benchmark/nmodl/test/benchmark/hh_complete_python_100mil/benchmark_results.pickle"]

In [ ]:
results = {}
for pickle_file in pickle_files:
    with open(pickle_file, 'rb') as handle:
        results.update(pickle.load(handle))
print(results)

In [ ]:
def _get_flags_string(flags):
    return flags.replace(" ", "_").replace('-','').replace('=','_')

In [ ]:
def generate_graph_pandas(compilers_comparison_config, graph_suffix, output_dir):
    df = None
    bar_data_state_cpu_panda = {}
    bar_data_state_cpu_panda["architecture"] = []
    bar_data_state_cpu_panda["compiler"] = []
    bar_data_state_cpu_panda["runtime"] = []
    bar_data_cur_cpu_panda = {}
    bar_data_cur_cpu_panda["architecture"] = []
    bar_data_cur_cpu_panda["compiler"] = []
    bar_data_cur_cpu_panda["runtime"] = []
    compiler_flags = json.loads(compilers_comparison_config)
    for modname in results:
        for architecture in results[modname]:
            bar_data_state_cpu_panda[architecture] = {}
            for compiler in results[modname][architecture]:
                if compiler in compiler_flags and architecture in compiler_flags[compiler]:
                    for flags in compiler_flags[compiler][architecture]:
                        dict_label = "{}_{}_{}".format(architecture, compiler, _get_flags_string(flags))
                        if compiler == "nmodl_jit":
                            state_kernel_name = "nrn_state_{}".format(modname.replace("-", "_"))
                            cur_kernel_name = "nrn_cur_{}".format(modname.replace("-", "_"))
                        else:
                            state_kernel_name = "nrn_state_ext"
                            cur_kernel_name = "nrn_cur_ext"
                        if compiler == "clang" and "jit" in flags:
                            bar_data_state_cpu_panda["architecture"].append(architecture)
                            bar_data_state_cpu_panda["compiler"].append("clang_jit")
                            bar_data_state_cpu_panda["runtime"].append(results[modname][architecture][compiler][_get_flags_string(flags)][state_kernel_name][0])
                            bar_data_cur_cpu_panda["architecture"].append(architecture)
                            bar_data_cur_cpu_panda["compiler"].append("clang_jit")
                            bar_data_cur_cpu_panda["runtime"].append(results[modname][architecture][compiler][_get_flags_string(flags)][cur_kernel_name][0])
                        else:
                            bar_data_state_cpu_panda["architecture"].append(architecture)
                            bar_data_state_cpu_panda["compiler"].append(compiler)
                            bar_data_state_cpu_panda["runtime"].append(results[modname][architecture][compiler][_get_flags_string(flags)][state_kernel_name][0])
                            bar_data_cur_cpu_panda["architecture"].append(architecture)
                            bar_data_cur_cpu_panda["compiler"].append(compiler)
                            bar_data_cur_cpu_panda["runtime"].append(results[modname][architecture][compiler][_get_flags_string(flags)][cur_kernel_name][0])
        df_state = pd.DataFrame(bar_data_state_cpu_panda, columns=["architecture", "compiler", "runtime"])
        sns.barplot(x='architecture', y='runtime', hue='compiler', data=df_state)
        plt.xlabel("Architecture")
        plt.ylabel("Runtime (s)")
        plt.title("nrn_state runtime for {}".format(modname))
        plt.savefig("{}/{}_state_benchmark_{}.pdf".format(output_dir, modname, graph_suffix), format="pdf", bbox_inches="tight")
        plt.show()
        plt.close()
        df_cur = pd.DataFrame(bar_data_cur_cpu_panda, columns=["architecture", "compiler", "runtime"])
        sns.barplot(x='architecture', y='runtime', hue='compiler', data=df_cur)
        plt.xlabel("Architecture")
        plt.ylabel("Runtime (s)")
        plt.title("nrn_cur runtime for {}".format(modname))
        plt.savefig("{}/{}_cur_benchmark_{}.pdf".format(output_dir, modname, graph_suffix), format="pdf", bbox_inches="tight")
        plt.show()
        plt.close()

compilers_comparison_config = """
{
  "intel": {
    "default": [
      "-O2 -prec-div"
    ],
    "nehalem": [
      "-O2 -msse2 -prec-div -fimf-use-svml"
    ],
    "broadwell": [
      "-O2 -march=broadwell -mtune=broadwell -prec-div -fimf-use-svml"
    ],
    "skylake-avx512": [
      "-O2 -march=skylake-avx512 -mtune=skylake -prec-div -fimf-use-svml"
    ]
  },
  "clang": {
    "default": [
      "-O3 -ffast-math",
      "-O3 -ffast-math jit"
    ],
    "nehalem": [
      "-O3 -march=nehalem -mtune=nehalem -ffast-math -fopenmp -fveclib=SVML",
      "-O3 -march=nehalem -mtune=nehalem -ffast-math -fopenmp -fveclib=SVML jit"
    ],
    "broadwell": [
      "-O3 -march=broadwell -mtune=broadwell -ffast-math -fopenmp -fveclib=SVML",
      "-O3 -march=broadwell -mtune=broadwell -ffast-math -fopenmp -fveclib=SVML jit"
    ],
    "skylake-avx512": [
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -fopenmp -fveclib=SVML",
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -fopenmp -fveclib=SVML jit"
    ]
  },
  "gcc": {
    "default": [
      "-O3 -ffast-math -ftree-vectorize"
    ],
    "nehalem": [
      "-O3 -march=nehalem -mtune=nehalem -ffast-math -ftree-vectorize -mveclibabi=svml -fopenmp"
    ],
    "broadwell": [
      "-O3 -march=broadwell -mtune=broadwell -ffast-math -ftree-vectorize -mveclibabi=svml -fopenmp"
    ],
    "skylake-avx512": [
      "-O3 -march=skylake-avx512 -mtune=skylake -ffast-math -ftree-vectorize -mveclibabi=svml -fopenmp"
    ]
  },
  "nmodl_jit": {
    "default": [
      "SVML_nnancontractafn"
    ],
    "nehalem": [
      "SVML_nnancontractafn"
    ],
    "broadwell": [
      "SVML_nnancontractafn"
    ],
    "skylake-avx512": [
      "SVML_nnancontractafn"
    ]
  }
}
"""

generate_graph_pandas(compilers_comparison_config, "cpu_all_compilers", "graphs_output_pandas")

In [ ]:
compilers_comparison_config = """
{
  "intel": {
    "default": [
      "-O2 -prec-div"
    ],
    "nehalem": [
      "-O2 -msse2 -prec-div -fimf-use-svml"
    ],
    "broadwell": [
      "-O2 -march=broadwell -mtune=broadwell -prec-div -fimf-use-svml"
    ],
    "skylake-avx512": [
      "-O2 -march=skylake-avx512 -mtune=skylake -prec-div -fimf-use-svml"
    ]
  },
  "nmodl_jit": {
    "default": [
      "SVML_nnancontractafn"
    ],
    "nehalem": [
      "SVML_nnancontractafn"
    ],
    "broadwell": [
      "SVML_nnancontractafn"
    ],
    "skylake-avx512": [
      "SVML_nnancontractafn"
    ]
  }
}
"""
generate_graph_pandas(compilers_comparison_config, "cpu_intel_vs_nmodl", "graphs_output_pandas")

In [ ]:
compilers_comparison_config = """
{
  "nvhpc": {
    "nvptx64": [
      "-O3 -gpu=nordc,fastmath"
    ]
  },
  "nmodl_jit": {
    "nvptx64": [
      "libdevice_nnancontractafn"
    ]
  }
}
"""
generate_graph_pandas(compilers_comparison_config, "gpu_openacc_vs_nmodl", "graphs_output_pandas")